In [2]:
import librosa
import pandas as pd
from librosa.effects import pitch_shift
import os
import pathlib
import random
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from librosa.feature import mfcc
import scipy.io.wavfile as wav
import tensorflow as tf
import scipy.signal as signal
import keras
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.optimizers import adam_v2
from keras.layers import Input, Conv1D, SeparableConv1D, MaxPooling1D, Flatten, Dense, Dropout, \
    BatchNormalization, Activation, LSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

In [3]:
numFeat=40
training_df=pd.read_csv("./range_training.csv")
y_train=training_df.loc[:,"Range"]
x_train=training_df.loc[:,"MFCC_0":"MFCC_39"]
x_train=np.array(x_train)
y_train=np.array(y_train)
mean = x_train.mean(axis=0)
x_train -= mean
std = x_train.std(axis=0)
x_train /= std
x_train=np.reshape(x_train, (len(x_train), numFeat, 1))


In [4]:
new_df= pd.DataFrame()
indiv_df=training_df.loc[training_df['Pass'] == "A1R1P1"]
new_df= pd.concat([new_df, indiv_df])
indiv_df=training_df.loc[training_df['Pass'] == "A1R1P2"]
new_df= pd.concat([new_df, indiv_df])
indiv_df=training_df.loc[training_df['Pass'] == "A1R1P3"]
new_df= pd.concat([new_df, indiv_df])
indiv_df=training_df.loc[training_df['Pass'] == "A1R1P4"]
new_df= pd.concat([new_df, indiv_df])
indiv_df=training_df.loc[training_df['Pass'] == "A1R1P5"]
new_df= pd.concat([new_df, indiv_df])
indiv_df=training_df.loc[training_df['Pass'] == "A2R1P1"]
new_df= pd.concat([new_df, indiv_df])
indiv_df=training_df.loc[training_df['Pass'] == "A2R1P2"]
new_df= pd.concat([new_df, indiv_df])
indiv_df=training_df.loc[training_df['Pass'] == "A2R1P3"]
new_df= pd.concat([new_df, indiv_df])
indiv_df=training_df.loc[training_df['Pass'] == "A2R1P4"]
new_df= pd.concat([new_df, indiv_df])
indiv_df=training_df.loc[training_df['Pass'] == "A2R1P5"]
new_df= pd.concat([new_df, indiv_df])
indiv_df=training_df.loc[training_df['Pass'] == "A2R2P1"]
new_df= pd.concat([new_df, indiv_df])
indiv_df=training_df.loc[training_df['Pass'] == "A2R2P2"]
new_df= pd.concat([new_df, indiv_df])
indiv_df=training_df.loc[training_df['Pass'] == "A2R2P3"]
new_df= pd.concat([new_df, indiv_df])
indiv_df=training_df.loc[training_df['Pass'] == "A2R2P4"]
new_df= pd.concat([new_df, indiv_df])
indiv_df=training_df.loc[training_df['Pass'] == "A2R2P5"]
new_df= pd.concat([new_df, indiv_df])
y_train=new_df.loc[:,"Range"]
x_train=new_df.loc[:,"MFCC_0":"MFCC_39"]
x_train=np.array(x_train)
y_train=np.array(y_train)
mean = x_train.mean(axis=0)
x_train -= mean
std = x_train.std(axis=0)
x_train /= std
x_train=np.reshape(x_train, (len(x_train), numFeat, 1))

In [5]:
trainFeatures, testFeatures, trainTargets, testTargets = train_test_split(x_train, y_train, test_size=0.9, random_state=42)

Attempt at a RCNN:

In [6]:

model =tf.keras.Sequential()
model.add(Conv1D(filters=32,kernel_size=3, strides=2, use_bias=False, name='conv_1',input_shape=(numFeat,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(3, strides=2))
model.add(Conv1D(filters=32,kernel_size=3, strides=2, use_bias=False,
                         name='conv_2'))
model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(MaxPooling1D(64, strides=2))
# hidden = MaxPooling1D(3, strides=2)(hidden)
model.add(Conv1D(filters=32,kernel_size=3, strides=2, use_bias=False, name='conv_3'))
model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(MaxPooling1D(3, strides=2))
model.add(LSTM(32))
model.add(Dense(64, activation='relu', name='dense_1'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu', name='dense_2'))
#model.add(LSTM(16))
model.add(Dense(128, activation='relu', name='dense_3'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear', name='dense_4'))


optimizer= tf.keras.optimizers.Adam() #method to prevent gradient clipping
model.compile(loss='mse',optimizer=optimizer,metrics=['MSE','MAE'])


reduceLR = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                 verbose=1, patience=10, mode='auto')
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-4,
                          patience=40, mode='auto', restore_best_weights=True)
model.summary()
history = model.fit(trainFeatures, trainTargets,
                      epochs=200,
                      batch_size=32,
                      shuffle=False,
                      validation_split=0.2,
                      callbacks=[reduceLR, early])
# hidden = MaxPooling1D(3, strides=2)(hidden)
#model.add(Flatten()) #this is where feature extraction turns to classification

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_1 (Conv1D)             (None, 19, 32)            96        
                                                                 
 batch_normalization (BatchN  (None, 19, 32)           128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 19, 32)            0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 9, 32)            0         
 )                                                               
                                                                 
 conv_2 (Conv1D)             (None, 4, 32)             3072      
                                                                 
 batch_normalization_1 (Batc  (None, 4, 32)            1

Attempt at a CNN:

In [23]:
inputs = Input(shape=(numFeat, 1))
hidden = Conv1D(filters=32,kernel_size=3, strides=2, use_bias=False, name='conv_1')(inputs)
hidden = BatchNormalization()(hidden)
hidden = Activation('relu')(hidden)
# hidden = MaxPooling1D(3, strides=2)(hidden)
hidden = Conv1D(filters=32,kernel_size=3, strides=2, use_bias=False,
                         name='conv_2')(hidden)
hidden = BatchNormalization()(hidden)
hidden = Activation('relu')(hidden)
# hidden = MaxPooling1D(3, strides=2)(hidden)
hidden = Conv1D(filters=32, kernel_size=3, strides=2, use_bias=False,
                         name='conv_3')(hidden)
hidden = BatchNormalization()(hidden)
hidden = Activation('relu')(hidden)
# hidden = MaxPooling1D(3, strides=2)(hidden)
hidden = Flatten()(hidden) #this is where feature extraction turns to classification
hidden = Dropout(0.5)(hidden)
hidden = Dense(864, activation='relu', name='dense_1')(hidden)  # 896, 864, 928
hidden = Dropout(0.5)(hidden)
dist = Dense(1, activation='linear', name='dense_2')(hidden)
network = Model(inputs=inputs, outputs=dist)

optimizer= tf.keras.optimizers.Adam() #method to prevent gradient clipping
network.compile(loss='mse',optimizer=optimizer,metrics=['MSE'])
reduceLR = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                 verbose=1, patience=10, mode='auto')
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-4,
                          patience=40, mode='auto', restore_best_weights=True)
network.summary()
history = network.fit(trainFeatures, trainTargets,
                      epochs=200,
                      batch_size=32,
                      shuffle=True,
                      validation_split=0.2,
                      callbacks=[reduceLR, early])

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 40, 1)]           0         
                                                                 
 conv_1 (Conv1D)             (None, 19, 32)            96        
                                                                 
 batch_normalization_33 (Bat  (None, 19, 32)           128       
 chNormalization)                                                
                                                                 
 activation_33 (Activation)  (None, 19, 32)            0         
                                                                 
 conv_2 (Conv1D)             (None, 9, 32)             3072      
                                                                 
 batch_normalization_34 (Bat  (None, 9, 32)            128       
 chNormalization)                                          

KeyboardInterrupt: 

Attempt at a ANN:

In [26]:
model =tf.keras.Sequential()
model.add(Dense(200,input_shape=(numFeat,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(864, activation='relu'))
#model.add(MaxPooling1D(64, strides=2))
# hidden = MaxPooling1D(3, strides=2)(hidden)
model.add(Dense(600, activation='relu'))
#model.add(MaxPooling1D(3, strides=2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))


optimizer= tf.keras.optimizers.Adam() #method to prevent gradient clipping
model.compile(loss='mse',optimizer=optimizer,metrics=['MSE','MAE'])


reduceLR = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                 verbose=1, patience=10, mode='auto')
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-4,
                          patience=40, mode='auto', restore_best_weights=True)
model.summary()
history = model.fit(trainFeatures, trainTargets,
                      epochs=200,
                      batch_size=32,
                      shuffle=True,
                      validation_split=0.2,
                      callbacks=[reduceLR, early])
# hidden = MaxPooling1D(3, strides=2)(hidden)
#model.add(Flatten()) #this is where feature extraction turns to classification

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 40, 200)           400       
                                                                 
 batch_normalization_37 (Bat  (None, 40, 200)          800       
 chNormalization)                                                
                                                                 
 activation_37 (Activation)  (None, 40, 200)           0         
                                                                 
 dense_4 (Dense)             (None, 40, 864)           173664    
                                                                 
 dense_5 (Dense)             (None, 40, 600)           519000    
                                                                 
 dense_6 (Dense)             (None, 40, 64)            38464     
                                                     

KeyboardInterrupt: 